In [33]:
from enum import Enum
from dotenv import  load_dotenv
load_dotenv()
from Models.models import LLMModel as LLMModel2
import pytest
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import (HumanMessage, AIMessage, FunctionMessage)
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field, validator
import json

In [34]:
class FunctionSchema(BaseModel):
    function: str = Field(..., description="function to run if any, set to 'None' if you are not running any function")
    args: dict = Field(..., description="dictionary representing parameter and values")
    agent_response: str = Field(..., description="Your final response after receiving functions output")
    agent_name: str = Field(..., description="Your name")
    directed_to: str = Field(..., description="who the message is directed to (human or agent_cody)")

    # @validator('function')
    # def validate_function(cls, v):
    #     if not isinstance(v, str):
    #         raise ValueError('function must be a string')
    #     return v

    # @validator('args')
    # def validate_args(cls, v):
    #     if not isinstance(v, list):
    #         raise ValueError('args must be a list')
    #     return v

    # @validator('ai_notes')
    # def validate_ai_notes(cls, v):
    #     if not isinstance(v, str):
    #         raise ValueError('ai_notes must be a string')
    #     return v


In [4]:
with open("/Users/daniel/Desktop/Projects/TURING/Turing_Evaluator/Data/python_basics_&_scripting__write_code_in_python__22__24_03_2024_08_57_45_10.ipynb", "r") as file:
    convo = file.read()

In [35]:
prompt_template = ChatPromptTemplate.from_messages([
    "system",
    "You are a helpful AI agent called agent_functions, that only communicates using JSON files and your job is to run any of the following functions. "
    "1. python_repl: takes in a parameter code, runs it and returns the result "
    "2. tavily_tool: takes in a topic, does a web search and returns the result "
    "You are working along side other AI agents:\n"
    "agent_cody execute the functions with arguments you have provided, and responds with the result"
    "finally you can give your final response"
])
prompt_template = prompt_template.partial(main_prompt=prompt_template)

In [38]:
model = "llama-v3-70b-instruct"
provider = "fireworks_api"


print(f"Testing model: {model}")
evaluator = LLMModel2(
    provider=provider,
    model=model,
    output_schema=FunctionSchema.model_json_schema(),
    name="aspect_evaluator",
    prompt_template=prompt_template,
    try_to_parse=True,
    config={
        "retry": 3,
    
    }
)





Testing model: llama-v3-70b-instruct


### CODE EXECUTOR

In [39]:
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union
from langchain_core.tools import tool
import contextlib
import io

@tool
def python_repl(code: Annotated[str, "The python code to execute."]):
    """Use this to execute python code when needed. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user and you.
    """
    # Create StringIO objects to capture stdout and stderr
    stdout = io.StringIO()
    stderr = io.StringIO()

    # Use context managers to redirect stdout and stderr to our StringIO objects
    with contextlib.redirect_stdout(stdout), contextlib.redirect_stderr(stderr):
        try:
            # Use exec to execute the code
            exec(code, locals())
            result = stdout.getvalue()
            error = stderr.getvalue()
        except Exception as e:
            # If an error occurs during execution, return the error message
            return f"Failed to execute. Error: {repr(e)}"

    # If no errors occurred, return the output
    if error:
        return f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}\nStderr: {error}"
    else:
        return f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"


all_tools = [

            python_repl,
        ]
tool_executor = ToolExecutor(all_tools)

def tool_node(message):
    tool_ = message
   
    action = ToolInvocation(
        tool=tool_["function"],
        tool_input=tool_["args"],
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = HumanMessage(
        content=f"{tool} response: {str(response)}", name=action.tool
    )
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [47]:
# Perform evaluation
user =  [
    HumanMessage("{'prompt': 'execute code to print 1 to 10 in python', 'agent_name':'human'}"),
    # AIMessage("{'function': 'python_repl', 'args': {'code': 'for i in range(1, 3):\n    print(i)\n'}, 'agent_name': 'agent_functions', 'agent_response': ''}"),
    # HumanMessage("{'function': 'None', 'args': {}, 'agent_name': 'agent_cody', 'agent_response': '1\n2\n3'}")
    ]

evaluation_result = evaluator(user)
evaluation_result

{'function': 'python_repl',
 'args': {'code': 'for i in range(1, 11):\n    print(i)'},
 'agent_response': '',
 'agent_name': 'agent_functions',
 'directed_to': 'agent_cody'}

In [48]:
tool_result = tool_node(evaluation_result)["messages"]
tool_result[0].content

'<function tool at 0x105d4d6c0> response: Successfully executed:\n```python\nfor i in range(1, 11):\n    print(i)\n```\nStdout: 1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n'

In [49]:
evaluator(tool_result)

{'function': 'None',
 'args': {},
 'agent_response': 'Code executed successfully. The output is: 1 2 3 4 5 6 7 8 9 10',
 'agent_name': 'agent_functions',
 'directed_to': 'human'}